In [1]:
import numpy as np
from numpy import random
import string
import math
from scipy.spatial import distance
import os

In [2]:
def gen_bounds():
    '''
    Get bounds for master gen
    '''

    x_l = random.randint(10, 90)
    x_h = random.randint(x_l+30, x_l+50)
    y_l = random.randint(10, 90)
    y_h = random.randint(y_l+30, y_l+50)

    return x_l, x_h, y_l, y_h

def gen_exponential(seedx, seedy):
    '''
    Generate exponential coordinates
    '''

    return random.exponential(seedx), random.exponential(seedy)

def gen_uniform(seedxl, seedxh, seedyl, seedyh):
    '''
    Generate uniform coordinates
    '''

    return int(random.uniform(seedxl, seedxh)), int(random.uniform(seedyl, seedyh))

def gen_master(x_l, x_h, y_l, y_h):
    '''
    Decide exponential or uniform
    '''

    if random.randint(1,2,1) > 1:
        return gen_exponential(random.uniform(x_l, x_h), random.uniform(y_l, y_h))
    else:
        return gen_uniform(x_l, x_h, y_l, y_h)

def gen_size():
    '''
    Generate data size
    '''

    return random.randint(20, 30)

def is_valid(coordinates, new_point, limit = 1):
    '''
    Is the new_point distant enough from others?
    limit? how to set - heuristic = 1
    '''

    for coord in coordinates:
        if np.linalg.norm(list((coord[0] - new_point[0], coord[1] - new_point[1]))) <= limit:
            return False
    return True

def gen_name(size = 7, chars=string.ascii_lowercase + string.ascii_uppercase + string.digits):
    '''
    Generate a random name for graph
    '''

    return ''.join(random.choice(list(chars)) for _ in range(size))

In [3]:
def main(howMany, adj_path='./adj', node_path='./node', random_seed=None):
    '''
    Generate random fully connected graphs
    '''
    
    if random_seed is not None:
        random.seed(random_seed)
        
    if adj_path == node_path:
        raise ValueError("adj_path and node_path must not be the same!")

    if not os.path.exists(adj_path):
        os.mkdir(adj_path)
        
    if not os.path.exists(node_path):
        os.mkdir(node_path)
        
    for i in range(howMany):
        graph = {}
        x_l, x_h, y_l, y_h = gen_bounds()
        graph_size = gen_size()
        graph_name = gen_name()
        nodes = []
        while len(nodes) < graph_size:
            coord = gen_master(x_l, x_h, y_l, y_h)
            if is_valid(nodes, coord):
                nodes.append(coord)

        adj = distance.cdist(nodes, nodes, 'euclidean')
        
        basename = graph_name+'_'+str(graph_size)
        adj_filename = os.path.join(adj_path, basename)
        node_filename = os.path.join(node_path, basename)
        np.savetxt(adj_filename, adj)
        np.savetxt(node_filename, nodes)


In [4]:
if __name__ == '__main__':
    main(100, './adj', './node', 2)